# Notebook 00: Setup e Preparazione Dati GTZAN

**Scopo:** Questo notebook ha il compito di caricare il dataset GTZAN originale, applicare tutto il pre-processing necessario (segmentazione, estrazione di spettrogrammi, normalizzazione) e salvare su disco un set di dati pulito e pronto per l'addestramento.

**Questo notebook va eseguito una sola volta.**

**Input:**
- Cartella con i file audio originali: `../data/gtzan/genres/`

**Output (salvati in `../data/processed/`):**
- Array NumPy: `X_train.npy`, `y_train.npy`, `X_val.npy`, `y_val.npy`, `X_test.npy`, `y_test.npy`
- Artefatti di pre-processing: `scaler.pkl`, `label_encoder.pkl`

In [18]:
# ===================================================================
# CELLA 1: SETUP, IMPORTS E CONFIGURAZIONE GLOBALE
# ===================================================================

import os
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle
from tqdm.notebook import tqdm

# --- Global Constants ---
DATA_PATH = '../data/gtzan/genres/'
PROCESSED_DATA_PATH = '../data/processed/'
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# --- Creazione Cartella di Output ---
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

print("✅ Ambiente pronto per la preparazione dei dati.")

✅ Ambiente pronto per la preparazione dei dati.


In [19]:
# ===================================================================
# CELLA 2: CLASSE DATALOADER (CORRETTA E POTENZIATA)
# ===================================================================

class GTZANDataLoader:
    """Carica, pre-processa e suddivide il dataset GTZAN."""
    
    def __init__(self, data_path, sample_rate=22050, n_mels=128, hop_length=512):
        self.data_path = data_path
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.hop_length = hop_length
        self.genres = sorted([d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))])
        self.label_encoder = LabelEncoder().fit(self.genres)
        self.scaler = StandardScaler()
        
    def load_file_paths(self):
        file_paths, labels = [], []
        for genre in self.genres:
            genre_path = os.path.join(self.data_path, genre)
            for filename in os.listdir(genre_path):
                if filename.endswith(('.wav', '.au')):
                    file_paths.append(os.path.join(genre_path, filename))
                    labels.append(genre)
        return file_paths, labels
    
    def process_file(self, file_path, n_segments=10, segment_duration=2.97):
        """Aumenta a 10 segmenti da ~3s per massimizzare la data augmentation."""
        try:
            signal, _ = librosa.load(file_path, sr=self.sample_rate)
            samples_per_segment = int(self.sample_rate * segment_duration)
            
            spectrograms = []
            for s in range(n_segments):
                start = s * samples_per_segment
                end = start + samples_per_segment
                if end <= len(signal):
                    segment_signal = signal[start:end]
                    mel_spec = librosa.feature.melspectrogram(
                        y=segment_signal, sr=self.sample_rate, n_mels=self.n_mels, hop_length=self.hop_length)
                    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
                    spectrograms.append(log_mel_spec)
            return spectrograms
        except Exception as e:
            print(f"Attenzione: Errore nel processare {os.path.basename(file_path)}: {e}")
            return []
            
    def create_dataset_from_files(self, file_paths, labels_text, n_segments=10):
        X_list, y_list = [], []
        encoded_labels = self.label_encoder.transform(labels_text)
        for i, file_path in enumerate(tqdm(file_paths, desc=f"Processing {len(file_paths)} files")):
            spectrograms = self.process_file(file_path, n_segments=n_segments)
            for spec in spectrograms:
                X_list.append(spec)
                y_list.append(encoded_labels[i])
        return X_list, np.array(y_list)

def adjust_spectrograms_shape(spec_list, target_len=128):
    """
    CORREZIONE: Ripristinato il corpo della funzione per risolvere il NameError.
    Uniforma gli spettrogrammi alla lunghezza target.
    """
    print(f"   - Uniformazione degli spettrogrammi alla lunghezza: {target_len} frame")
    adjusted_list = []
    for spec in spec_list:
        if spec.shape[1] > target_len:
            adjusted_list.append(spec[:, :target_len])
        else:
            padding = target_len - spec.shape[1]
            adjusted_list.append(np.pad(spec, ((0, 0), (0, padding)), mode='constant'))
    return np.array(adjusted_list)

print("✅ Classe GTZANDataLoader definita e corretta.")

✅ Classe GTZANDataLoader definita e corretta.


In [20]:
# ===================================================================
# CELLA 3: ESECUZIONE E SALVATAGGIO
# ===================================================================

# 1. Caricamento percorsi e suddivisione
data_loader = GTZANDataLoader(data_path=DATA_PATH)
file_paths, labels_text = data_loader.load_file_paths()

train_files, test_files, train_labels_text, test_labels_text = train_test_split(
    file_paths, labels_text, test_size=0.2, random_state=RANDOM_STATE, stratify=labels_text)
train_files, val_files, train_labels_text, val_labels_text = train_test_split(
    train_files, train_labels_text, test_size=0.25, random_state=RANDOM_STATE, stratify=train_labels_text)
# 2. Estrazione spettrogrammi
X_train_list, y_train = data_loader.create_dataset_from_files(train_files, train_labels_text)
X_val_list, y_val = data_loader.create_dataset_from_files(val_files, val_labels_text)
X_test_list, y_test = data_loader.create_dataset_from_files(test_files, test_labels_text)

# 3. Uniformazione shape
X_train = adjust_spectrograms_shape(X_train_list)
X_val = adjust_spectrograms_shape(X_val_list)
X_test = adjust_spectrograms_shape(X_test_list)

# 4. Normalizzazione
scaler = data_loader.scaler
X_train_shape = X_train.shape
X_train = scaler.fit_transform(X_train.reshape(-1, X_train_shape[1] * X_train_shape[2])).reshape(X_train_shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[1] * X_val.shape[2])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])).reshape(X_test.shape)

# 5. Aggiunta canale
X_train, X_val, X_test = X_train[..., np.newaxis], X_val[..., np.newaxis], X_test[..., np.newaxis]

# --- SALVATAGGIO DEGLI ARTEFATTI ---
print("\n💾 Salvataggio dei dati processati e degli artefatti...")
np.save(os.path.join(PROCESSED_DATA_PATH, 'X_train.npy'), X_train)
np.save(os.path.join(PROCESSED_DATA_PATH, 'y_train.npy'), y_train)
np.save(os.path.join(PROCESSED_DATA_PATH, 'X_val.npy'), X_val)
np.save(os.path.join(PROCESSED_DATA_PATH, 'y_val.npy'), y_val)
np.save(os.path.join(PROCESSED_DATA_PATH, 'X_test.npy'), X_test)
np.save(os.path.join(PROCESSED_DATA_PATH, 'y_test.npy'), y_test)

with open(os.path.join(PROCESSED_DATA_PATH, 'scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)
with open(os.path.join(PROCESSED_DATA_PATH, 'label_encoder.pkl'), 'wb') as f:
    pickle.dump(data_loader.label_encoder, f)
print("\n✅ Processo completato. I dati sono pronti per il training nel notebook '01_Model_Training'.")
print("\n📊 Riepilogo Dati Salvati:")
print(f"   - Training Set:   X={X_train.shape}, y={y_train.shape}")
print(f"   - Validation Set: X={X_val.shape}, y={y_val.shape}")
print(f"   - Test Set:       X={X_test.shape}, y={y_test.shape}")

Processing 600 files:   0%|          | 0/600 [00:00<?, ?it/s]

/tmp/ipykernel_24226/2071647414.py:30: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, _ = librosa.load(file_path, sr=self.sample_rate)
/home/alepot55/Desktop/projects/naml_project/venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Attenzione: Errore nel processare jazz.00054.wav: 


Processing 200 files:   0%|          | 0/200 [00:00<?, ?it/s]

Processing 200 files:   0%|          | 0/200 [00:00<?, ?it/s]

   - Uniformazione degli spettrogrammi alla lunghezza: 128 frame
   - Uniformazione degli spettrogrammi alla lunghezza: 128 frame
   - Uniformazione degli spettrogrammi alla lunghezza: 128 frame

💾 Salvataggio dei dati processati e degli artefatti...

✅ Processo completato. I dati sono pronti per il training nel notebook '01_Model_Training'.

📊 Riepilogo Dati Salvati:
   - Training Set:   X=(5990, 128, 128, 1), y=(5990,)
   - Validation Set: X=(2000, 128, 128, 1), y=(2000,)
   - Test Set:       X=(2000, 128, 128, 1), y=(2000,)
